In [1]:
import asyncio
from nt import system
from langchain_core.prompts import prompt
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.constants import END, START
from typing import Annotated, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph import StateGraph
from langgraph.config import get_stream_writer 
from langgraph.checkpoint.memory import InMemorySaver
import operator  # 添加这个重要导入

from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
import sys


# 系统中所有可用的节点类型
nodes = ["supervisor", "triage", "recommend", "experts", "other"]

# 初始化大语言模型，使用通义千问2.5模型
 


分诊智能体的初步验证
输入：对患者症状的描述
输出：分诊级别、分诊科室、追问的问题

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
# 初始化 LLM
import json

llm = ChatOpenAI(
    model="qwen2.5:14b",
    base_url="https://zjlchat.vip.cpolar.cn/v1",
    api_key="EMPTY",
    temperature=0.1,
    top_p=0.8
)


# 初始化 MCP 客户端
client = MultiServerMCPClient({
            "triage": {
                "url": "http://localhost:8000/sse",
                "headers": {},
                "transport": "sse"
            }
        })

# 获取工具
tools = await client.get_tools()
print(f"成功加载 {len(tools)} 个工具")

# 创建智能体
triage1 = create_react_agent(
    model=llm,
    tools=tools,
)

# 调用智能体
response = await triage1.ainvoke({
    "messages": [
        {"role":"system","content":"""
你是一个专业的医学顾问。请严格遵循以下流程进行问诊：

【流程说明】
1. 风险因素获取：根据医生对患者描述的症状，使用工具获取相关的医学风险因素
2. 针对性提问：基于获得的风险因素，生成具体的问诊问题

【输出格式要求】
请严格按照以下模板输出：

**风险因素确认问诊：**

[问题列表]

---
**格式规范：**
- 每个问题必须以数字序号开头（1. 2. 3. ...）
- 每个问题必须使用"患者是否有[风险因素]病史？"或"患者是否存在[风险因素]？"的句式
- 问题数量与风险因素数量严格对应
- 问题排列顺序按照风险因素的重要性从高到低

【示例】
输入风险因素：["糖尿病", "高血压", "冠心病"]
输出：
**风险因素确认问诊：**

1. 患者是否有糖尿病病史？
2. 患者是否有高血压病史？
3. 患者是否有冠心病病史？

【注意事项】
- 仅基于工具获取的风险因素生成问题
- 不得引入外部医学知识或主观推断
- 保持问题简洁专业，聚焦于病史和风险因素确认
- 使用中文进行提问，表述清晰易懂
        
        """},
        {"role": "user", "content": "皮肤红肿，发热"}
    ]
})

response["messages"][-1].content


成功加载 1 个工具


'**风险因素确认问诊：**\n\n1. 患者是否有皮肤软组织损伤病史？\n2. 患者是否存在慢性肝病？\n3. 患者是否有糖尿病病史？\n4. 患者的年龄是否≥60岁？'

In [ ]:
from langgraph.prebuilt import create_react_agent 
from langgraph.checkpoint.memory import InMemorySaver
from qwen_agent.llm import get_chat_model
from langchain_openai import ChatOpenAI
import datetime
checkpointer = InMemorySaver()



def get_current_date():
    """获取当前日期，返回格式为 YYYY-MM-DD"""
    return datetime.datetime.today().strftime("%Y-%m-%d")


   
   # 创建 LangChain 兼容的模型
model = ChatOpenAI(
       model="qwen2.5:14b",
       base_url="https://zjlchat.vip.cpolar.cn/v1",
       api_key="EMPTY",
       temperature=0.1,
       top_p=0.8
   )


 

agent = create_react_agent(
       model=model,
       tools=[get_current_date],
   )



triage_prompt="""【系统角色与核心指令】
你是一名专业的急诊分诊AI助手。你的任务是：仅根据用户输入的一段患者症状描述，进行一次性、非诊断性的分诊评估。 你必须直接输出结构化的分诊建议，包含分诊级别和建议科室，全程不得向用户提问或要求更多信息。

【分析逻辑与判断标准】
你将遵循以下步骤进行内部分析：

信息提取与风险评估：

迅速扫描症状描述，识别所有危重指征。

关键危重指征（Ⅰ级/Ⅱ级）包括但不限于：

生命威胁：心跳呼吸骤停、窒息、严重呼吸困难、昏迷、无反应、大动脉活动性出血。

严重症状：剧烈胸痛并放射至左臂/后背、突发偏瘫/口眼歪斜/言语不清、严重过敏反应（喉头水肿、休克）、抽搐持续状态、剧烈头痛伴呕吐/意识改变。

高危情况：休克表现（面色苍白、湿冷、脉搏微弱）、严重创伤（多发伤、高位脊髓损伤）、体温＞41℃或＜35℃。

确定分诊级别：

根据上述危重指征，遵循“就高不就低”原则，将患者归入最高匹配的级别。

Ⅰ级（濒危）：存在任何一项生命威胁指征。

Ⅱ级（危重）：存在任何一项严重症状或高危情况指征，但暂无即刻的生命停止风险。

Ⅲ级（紧急）：病情稳定但可能恶化。例如：高热伴寒战、中度腹痛、肾绞痛、哮喘发作（用药后可部分缓解）、开放性骨折但出血已控制。

Ⅳ级（次紧急）：病情稳定，急性发作。例如：轻度发热、呕吐腹泻无脱水、扭伤、皮疹、尿路感染。

Ⅴ级（非紧急）：慢性问题或轻微症状。例如：慢性病复查、轻微感冒、健康咨询。

推荐就诊科室：

基于症状的主要部位和性质进行判断。

急诊内科：发热、感染、腹痛、呕吐腹泻、胸痛、呼吸困难、头晕、头痛、意识问题、中毒等。

急诊外科：各类外伤、撞击伤、切割伤、烧伤、动物咬伤、急性腹痛（怀疑阑尾炎、肠梗阻、穿孔等）。

急诊骨科：骨折、脱位、扭伤、拉伤、背部或肢体急性疼痛与活动受限。

神经内科/急诊神经科：突发头痛、眩晕、抽搐、偏瘫、麻木、言语不清、面瘫。

其他：根据具体情况，也可能建议儿科急诊（儿童患者）、心血管内科（疑似心绞痛）、皮肤科（主要问题为皮疹）等。

【输出模板】
你必须严格使用以下格式输出，不添加任何其他内容：

分诊级别： [填写对应的罗马数字级别，后跟中文说明]
建议科室： [填写最优先的1-2个科室]
核心依据： [用简短的1-2句话说明为何定为此级别和科室，引用输入中的关键症状]"""


prompt=[
    {"role": "system", "content": triage_prompt},
    {"role": "user", "content":"皮肤红肿发热"},
]
response2=llm.invoke(prompt)
response2.content


'分诊级别： Ⅴ级（非紧急）\n建议科室： 皮肤科\n核心依据： 患者描述的症状为局部皮肤问题，无生命威胁或严重系统性表现。'

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
import asyncio

# 定义状态
class ParallelState(TypedDict):
    user_input: str
    triage1_result: str
    triage2_result: str
    combined_analysis: str

# 初始化 LLM
llm = ChatOpenAI(
    model="qwen2.5:14b",
    base_url="https://zjlchat.vip.cpolar.cn/v1",
    api_key="EMPTY",
    temperature=0.1,
    top_p=0.8
)

# 初始化 MCP 客户端
client = MultiServerMCPClient({
    "triage": {
        "url": "http://localhost:8000/sse",
        "headers": {},
        "transport": "sse"
    }
})

# 获取工具
tools = await client.get_tools()
print(f"成功加载 {len(tools)} 个工具")

# 创建第一个智能体（医学顾问）
triage1 = create_react_agent(
    model=llm,
    tools=tools,
)

# 第二个智能体的提示词
triage_prompt = """【系统角色与核心指令】
你是一名专业的急诊分诊AI助手。你的任务是：仅根据用户输入的一段患者症状描述，进行一次性、非诊断性的分诊评估。你必须直接输出结构化的分诊建议，包含分诊级别和建议科室，全程不得向用户提问或要求更多信息。

【分析逻辑与判断标准】
你将遵循以下步骤进行内部分析：

信息提取与风险评估：
迅速扫描症状描述，识别所有危重指征。

关键危重指征（Ⅰ级/Ⅱ级）包括但不限于：
生命威胁：心跳呼吸骤停、窒息、严重呼吸困难、昏迷、无反应、大动脉活动性出血。
严重症状：剧烈胸痛并放射至左臂/后背、突发偏瘫/口眼歪斜/言语不清、严重过敏反应（喉头水肿、休克）、抽搐持续状态、剧烈头痛伴呕吐/意识改变。
高危情况：休克表现（面色苍白、湿冷、脉搏微弱）、严重创伤（多发伤、高位脊髓损伤）、体温＞41℃或＜35℃。

确定分诊级别：
根据上述危重指征，遵循"就高不就低"原则，将患者归入最高匹配的级别。

Ⅰ级（濒危）：存在任何一项生命威胁指征。
Ⅱ级（危重）：存在任何一项严重症状或高危情况指征，但暂无即刻的生命停止风险。
Ⅲ级（紧急）：病情稳定但可能恶化。例如：高热伴寒战、中度腹痛、肾绞痛、哮喘发作（用药后可部分缓解）、开放性骨折但出血已控制。
Ⅳ级（次紧急）：病情稳定，急性发作。例如：轻度发热、呕吐腹泻无脱水、扭伤、皮疹、尿路感染。
Ⅴ级（非紧急）：慢性问题或轻微症状。例如：慢性病复查、轻微感冒、健康咨询。

推荐就诊科室：
基于症状的主要部位和性质进行判断。

急诊内科：发热、感染、腹痛、呕吐腹泻、胸痛、呼吸困难、头晕、头痛、意识问题、中毒等。
急诊外科：各类外伤、撞击伤、切割伤、烧伤、动物咬伤、急性腹痛（怀疑阑尾炎、肠梗阻、穿孔等）。
急诊骨科：骨折、脱位、扭伤、拉伤、背部或肢体急性疼痛与活动受限。
神经内科/急诊神经科：突发头痛、眩晕、抽搐、偏瘫、麻木、言语不清、面瘫。
其他：根据具体情况，也可能建议儿科急诊（儿童患者）、心血管内科（疑似心绞痛）、皮肤科（主要问题为皮疹）等。

【输出模板】
你必须严格使用以下格式输出，不添加任何其他内容：

分诊级别： [填写对应的罗马数字级别，后跟中文说明]
建议科室： [填写最优先的1-2个科室]
核心依据： [用简短的1-2句话说明为何定为此级别和科室，引用输入中的关键症状]"""

# 定义第一个智能体节点（医学顾问）
async def triage1_node(state: ParallelState) -> ParallelState:
    """第一个智能体：医学顾问，负责风险因素确认问诊"""
    
    response = await triage1.ainvoke({
        "messages": [
            {"role":"system","content":"""
你是一个专业的医学顾问。请严格遵循以下流程进行问诊：

【流程说明】
1. 风险因素获取：根据医生对患者描述的症状，使用工具获取相关的医学风险因素
2. 针对性提问：基于获得的风险因素，生成具体的问诊问题

【输出格式要求】
请严格按照以下模板输出：

**风险因素确认问诊：**

[问题列表]

---
**格式规范：**
- 每个问题必须以数字序号开头（1. 2. 3. ...）
- 每个问题必须使用"患者是否有[风险因素]病史？"或"患者是否存在[风险因素]？"的句式
- 问题数量与风险因素数量严格对应
- 问题排列顺序按照风险因素的重要性从高到低

【示例】
输入风险因素：["糖尿病", "高血压", "冠心病"]
输出：
**风险因素确认问诊：**

1. 患者是否有糖尿病病史？
2. 患者是否有高血压病史？
3. 患者是否有冠心病病史？

【注意事项】
- 仅基于工具获取的风险因素生成问题
- 不得引入外部医学知识或主观推断
- 保持问题简洁专业，聚焦于病史和风险因素确认
- 使用中文进行提问，表述清晰易懂
            """},
            {"role": "user", "content": state["user_input"]}
        ]
    })
    
    return {"triage1_result": response["messages"][-1].content}

# 定义第二个智能体节点（急诊分诊）
async def triage2_node(state: ParallelState) -> ParallelState:
    """第二个智能体：急诊分诊，负责分诊评估"""
    
    prompt = [
        {"role": "system", "content": triage_prompt},
        {"role": "user", "content": state["user_input"]},
    ]
    
    response = await llm.ainvoke(prompt)
    return {"triage2_result": response.content}

# 定义结果合并节点
def combine_results(state: ParallelState) -> ParallelState:
    """合并两个智能体的分析结果"""
    
    combined = f"""# 医学分析报告

## 1. 风险因素确认问诊
{state['triage1_result']}

## 2. 急诊分诊评估
{state['triage2_result']}

## 3. 综合分析建议
基于以上两个维度的分析：
- 医学顾问已提供需要确认的关键风险因素
- 分诊系统已评估紧急程度和就诊科室
请结合两者信息进行综合诊断决策。"""
    
    return {"combined_analysis": combined}

# 构建图
def create_parallel_triage_graph():
    """创建并行分诊图"""
    
    workflow = StateGraph(ParallelState)
    
    # 添加节点
    workflow.add_node("triage1", triage1_node)  # 医学顾问
    workflow.add_node("triage2", triage2_node)  # 急诊分诊
    workflow.add_node("combine", combine_results)  # 结果合并
    
    # 设置入口点
    workflow.set_entry_point("triage1")
    
    # 添加并行边
    workflow.add_edge("triage1", "triage2")
    workflow.add_edge("triage2", "combine")
    workflow.add_edge("combine", END)
    
    return workflow.compile()

# 使用示例
async def main():
    # 创建图
    graph = create_parallel_triage_graph()
    
    # 准备输入
    user_input = "皮肤红肿，发热"
    initial_state = {
        "user_input": user_input,
        "triage1_result": "",
        "triage2_result": "", 
        "combined_analysis": ""
    }
    
    # 执行图
    print("开始并行分析...")
    final_state = await graph.ainvoke(initial_state)
    
    # 输出结果
    print("\n" + "="*50)
    print("最终分析结果:")
    print("="*50)
    print(final_state["combined_analysis"])
    
    return final_state

# 运行
if __name__ == "__main__":
    result = await main()

成功加载 1 个工具
开始并行分析...

最终分析结果:
# 医学分析报告

## 1. 风险因素确认问诊
**风险因素确认问诊：**

1. 患者是否有皮肤软组织损伤病史？
2. 患者是否存在慢性肝病？
3. 患者是否有糖尿病病史？
4. 患者的年龄是否≥60岁？

以上问题基于患者描述的症状“皮肤红肿，发热”进行分析得出。

## 2. 急诊分诊评估
分诊级别： Ⅲ级（紧急）
建议科室： 急诊内科
核心依据： 发热伴局部红肿提示可能有感染或其他急性炎症反应。

## 3. 综合分析建议
基于以上两个维度的分析：
- 医学顾问已提供需要确认的关键风险因素
- 分诊系统已评估紧急程度和就诊科室
请结合两者信息进行综合诊断决策。


: 

推荐检查节点

In [ ]:
from langgraph.prebuilt import create_react_agent 
from langgraph.checkpoint.memory import InMemorySaver
from qwen_agent.llm import get_chat_model
from langchain_openai import ChatOpenAI
import datetime


import math
import numpy as np

def analyze_disease_probability(disease_data, risk_factor_count):
    """
    分析疾病概率并返回最可能的疾病
    
    参数:
    disease_data: dict, 格式为 {"疾病名称": 得分, ...}
    risk_factor_count: int, 风险因素总数
    
    返回:
    dict: 包含最高概率疾病信息和softmax结果的字典
    """
    
    # 提取疾病名称和得分
    disease_names = list(disease_data.keys())
    disease_scores = list(disease_data.values())
    
    # 计算softmax概率
    exp_scores = [math.exp(score) for score in disease_scores]
    sum_exp_scores = sum(exp_scores)
    probabilities = [exp_score / sum_exp_scores for exp_score in exp_scores]
    
    # 找到最高概率的疾病
    max_prob_index = np.argmax(probabilities)
    most_likely_disease = disease_names[max_prob_index]
    confidence = probabilities[max_prob_index]
    
    # 构建结果
    result = {
        "most_likely_disease": most_likely_disease,
        "confidence": round(confidence * 100, 2),  # 转换为百分比，保留2位小数
        "disease_details": {},
        "risk_factor_count": risk_factor_count
    }
    
    # 为每个疾病添加详细信息
    for i, disease in enumerate(disease_names):
        result["disease_details"][disease] = {
            "score": disease_scores[i],
            "probability": round(probabilities[i] * 100, 2)
        }
    
    return result

def query_disease_info(disease_name):
    """
    使用Cypher查询疾病详细信息（待填充）
    
    参数:
    disease_name: str, 疾病名称
    
    返回:
    dict: 查询结果（目前返回空字典，待您填充Cypher查询逻辑）
    """
    # TODO: 在这里添加您的Cypher查询代码
    # 示例结构：
    # cypher_query = f"MATCH (d:Disease {{name: '{disease_name}'}}) RETURN d"
    # 执行查询并返回结果
    
    # 临时返回空结果
    return {
        "disease_name": disease_name,
        "additional_info": "Cypher查询结果将在这里显示",
        "query_status": "待实现"
    }

# 使用示例
 
   
   # 创建 LangChain 兼容的模型
model = ChatOpenAI(
       model="qwen2.5:14b",
       base_url="https://zjlchat.vip.cpolar.cn/v1",
       api_key="EMPTY",
       temperature=0.1,
       top_p=0.8
   )


 

recommend = create_react_agent(
       model=model,
       tools=[get_current_date],
   )



prompt="""
你是一名医学症状对比分析师。你的任务是根据患者的回答，与给定的疾病及其风险因素列表进行逐一比对，并严格遵循以下规则进行计算。

【任务步骤】

疾病可能性得分计算：针对每个疾病下的每一个风险因素问题，根据患者的回答进行判断。

回答为“是”或症状存在：该疾病可能性+1。

回答为“否”或症状不存在：该疾病可能性+0。

患者的回答中未提及该因素：该疾病可能性+0。

风险因素总数计算：统计所有疾病下列出的不重复的风险因素总数量。即使同一因素出现在不同疾病中，也只计一次。

输出：最终，你只需要输出每种疾病的名称及其得分，以及不重复的风险因素总数。请严格使用“疾病名称:得分”的格式，并使用阿拉伯数字。最终输出格式为：“疾病名称1:得分, 疾病名称2:得分, ..., 风险因素总数:总数”。

【示例】
给定疾病范围：
【可能疾病1】支气管哮喘急性发作

吸烟：患者是否有吸烟病史？

过敏原吸入：患者是否存在接触尘螨、花粉、宠物皮屑等过敏原的情况？

呼吸道感染：患者近期是否出现过呼吸道感染？

【可能疾病2】坏死性软组织感染

皮肤软组织损伤：患者是否有皮肤或软组织的创伤或挤压伤历史？

慢性肝病：患者是否有慢性肝病史？

糖尿病：患者是否存在糖尿病？

高龄：患者年龄是否≥60岁？

患者回答： 有吸烟史、没有过敏现象、没有呼吸道感染、年龄66岁、有糖尿病、没有皮肤损伤。

分析过程：

支气管哮喘急性发作：吸烟(是→+1)，过敏原吸入(否→+0)，呼吸道感染(否→+0)。得分：1

坏死性软组织感染：皮肤损伤(否→+0)，慢性肝病(未提及→+0)，糖尿病(是→+1)，高龄(是→+1)。得分：2

风险因素总数：总共有【吸烟、过敏原吸入、呼吸道感染、皮肤损伤、慢性肝病、糖尿病、高龄】7个不重复因素。总数：7

输出： 支气管哮喘急性发作:1, 坏死性软组织感染:2, 风险因素总数:7



""" 